# PACE Data Preprocessor for Fishing Potential Correlation

Este notebook pré-processa dados PACE (Plankton, Aerosol, Cloud, ocean Ecosystem) para gerar variáveis derivadas relevantes para predição de potencial pesqueiro, conforme métricas descritas no relatório técnico.

## Variáveis derivadas:
- **chlor_a**: Clorofila-a [mg m⁻³]
- **carbon_phyto**: Carbono Fitoplanctônico C_phyto [mg m⁻³]
- **chl_c_ratio**: Razão Chl:C - indicador de taxa de crescimento μ
- **bbp_s**: Inclinação espectral η do backscattering - proxy para PSD (tamanho de partículas)
- **poc**: Carbono Orgânico Particulado [mg m⁻³]
- **Kd_490**: Coeficiente de atenuação difusa em 490nm [m⁻¹] - para calcular Z_eu

## Estratégias de preenchimento temporal:
1. **strict**: Valor exato do dia - NaN permanece NaN
2. **filled**: Busca valor válido em janela ±4 dias (prioridade: dia atual > mais próximo > passado em empate)

In [ ]:
# Cell 1: Instalação de dependências
import sys, subprocess
pkgs = ["pandas", "numpy", "xarray", "netCDF4", "scipy"]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + pkgs + ["--quiet"])
print("Dependências instaladas!")

In [ ]:
# Cell 2: Imports
import os
import re
import warnings
from pathlib import Path
from datetime import datetime, timedelta
from collections import defaultdict
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import RegularGridInterpolator

warnings.filterwarnings('ignore')
print(f"OK - numpy {np.__version__}, pandas {pd.__version__}, xarray {xr.__version__}")

In [ ]:
# Cell 3: Configuração

# =============================================================================
# CONFIGURAÇÃO - EDITAR CONFORME NECESSÁRIO
# =============================================================================

# Diretório com arquivos PACE brutos (subpastas por produto)
PACE_DATA_DIR = Path("../../data/pace")

# Diretório de saída para NetCDFs processados
OUTPUT_DIR = Path("./data/pace_processed")

# Janela temporal para preenchimento de NaN (dias antes/depois)
TEMPORAL_WINDOW = 4

# Mapeamento de produtos PACE para seus arquivos
# Padrão esperado: pace_{product}_{YYYYMMDD}.nc
PACE_PRODUCTS = {
    'carbon': {'var': 'carbon_phyto', 'output_name': 'carbon_phyto'},
    'chl': {'var': 'chlor_a', 'output_name': 'chlor_a'},
    'iop': {'var': 'bbp_s', 'output_name': 'bbp_s'},  # η - inclinação espectral
    'poc': {'var': 'poc', 'output_name': 'poc'},
    'kd': {'var': 'Kd', 'wavelength_idx': None, 'output_name': 'Kd_490'},  # Kd tem dimensão wavelength
    'rrs': {'var': 'Rrs', 'wavelength_idx': None, 'output_name': 'Rrs'},  # Para possíveis derivadas futuras
}

# Comprimentos de onda de interesse para Kd (490nm é o mais comum para Z_eu)
KD_TARGET_WAVELENGTH = 490  # nm

print(f"Diretório PACE: {PACE_DATA_DIR}")
print(f"Diretório de saída: {OUTPUT_DIR}")
print(f"Janela temporal: ±{TEMPORAL_WINDOW} dias")

In [ ]:
# Cell 4: Funções de descoberta de arquivos

def discover_pace_files(data_dir: Path) -> Dict[str, Dict[datetime, Path]]:
    """
    Descobre arquivos PACE organizados por produto e data.
    
    Espera estrutura:
    data_dir/
      pace_carbon_20250328.nc
      pace_chl_20250328.nc
      ...
    
    Retorna:
        Dict[produto -> Dict[data -> arquivo]]
    """
    files_by_product = defaultdict(dict)
    
    if not data_dir.exists():
        print(f"AVISO: Diretório {data_dir} não existe!")
        return files_by_product
    
    # Padrão: pace_{produto}_{YYYYMMDD}.nc
    pattern = re.compile(r'pace_([a-z]+)_(\d{8})\.nc', re.IGNORECASE)
    
    for f in data_dir.glob('*.nc'):
        match = pattern.match(f.name)
        if match:
            product = match.group(1).lower()
            date_str = match.group(2)
            try:
                date = datetime.strptime(date_str, '%Y%m%d')
                files_by_product[product][date] = f
            except ValueError:
                continue
    
    # Resumo
    print("\n" + "="*60)
    print("ARQUIVOS PACE DESCOBERTOS")
    print("="*60)
    for product, dates in sorted(files_by_product.items()):
        date_range = f"{min(dates).strftime('%Y-%m-%d')} a {max(dates).strftime('%Y-%m-%d')}"
        print(f"  {product:12s}: {len(dates):4d} arquivos ({date_range})")
    print("="*60)
    
    return dict(files_by_product)


def get_available_dates(files_by_product: Dict) -> List[datetime]:
    """Retorna lista de datas que têm pelo menos um produto disponível."""
    all_dates = set()
    for dates in files_by_product.values():
        all_dates.update(dates.keys())
    return sorted(all_dates)


# Testar descoberta
pace_files = discover_pace_files(PACE_DATA_DIR)

In [ ]:
# Cell 5: Funções de leitura de variáveis PACE

def read_pace_variable(filepath: Path, var_name: str, 
                       wavelength_target: Optional[int] = None) -> Tuple[Optional[np.ndarray], np.ndarray, np.ndarray]:
    """
    Lê uma variável de um arquivo PACE.
    
    Args:
        filepath: Caminho do arquivo NetCDF
        var_name: Nome da variável
        wavelength_target: Se a variável tem dimensão wavelength, extrai este comprimento de onda
    
    Returns:
        (data_2d, lats, lons) ou (None, None, None) se erro
    """
    try:
        with xr.open_dataset(filepath) as ds:
            if var_name not in ds.data_vars:
                return None, None, None
            
            data = ds[var_name]
            lats = ds['lat'].values
            lons = ds['lon'].values
            
            # Se tem dimensão wavelength, extrair banda específica
            if 'wavelength' in data.dims and wavelength_target is not None:
                wavelengths = ds['wavelength'].values
                # Encontrar índice mais próximo
                idx = np.argmin(np.abs(wavelengths - wavelength_target))
                actual_wl = wavelengths[idx]
                data = data.isel(wavelength=idx)
                # print(f"  → Extraído wavelength={actual_wl}nm (alvo: {wavelength_target}nm)")
            
            # Converter para numpy e aplicar scale/offset se necessário
            values = data.values.astype(np.float32)
            
            # Tratar fill values
            fill_val = data.attrs.get('_FillValue', -32767)
            values = np.where(values == fill_val, np.nan, values)
            values = np.where(np.abs(values) > 1e10, np.nan, values)
            
            return values, lats, lons
            
    except Exception as e:
        print(f"  ERRO lendo {filepath}: {e}")
        return None, None, None


def get_kd_wavelength_index(filepath: Path, target_wl: int = 490) -> Optional[int]:
    """Obtém o índice do comprimento de onda mais próximo do alvo para Kd."""
    try:
        with xr.open_dataset(filepath) as ds:
            if 'wavelength' in ds.dims:
                wls = ds['wavelength'].values
                idx = np.argmin(np.abs(wls - target_wl))
                return int(idx), int(wls[idx])
    except:
        pass
    return None, None


print("Funções de leitura definidas.")

In [ ]:
# Cell 6: Funções de preenchimento temporal

def find_nearest_valid_date(target_date: datetime, 
                            available_dates: Dict[datetime, Path],
                            window: int = 4) -> Optional[datetime]:
    """
    Encontra a data válida mais próxima dentro da janela temporal.
    
    Regra de prioridade:
    1. Se target_date tem dados, retorna ela
    2. Procura a data mais próxima em [target-window, target+window]
    3. Em caso de empate de distância, prefere o passado
    
    Args:
        target_date: Data alvo
        available_dates: Dict de datas disponíveis -> arquivos
        window: Janela em dias
    
    Returns:
        Data mais próxima ou None se não encontrar
    """
    if target_date in available_dates:
        return target_date
    
    candidates = []
    for dt in available_dates:
        delta = (dt - target_date).days
        if -window <= delta <= window:
            candidates.append((abs(delta), delta, dt))
    
    if not candidates:
        return None
    
    # Ordenar por: distância absoluta, depois por delta (negativo = passado = preferido)
    candidates.sort(key=lambda x: (x[0], x[1]))
    return candidates[0][2]


class PACEDataLoader:
    """
    Carrega dados PACE com cache e preenchimento temporal.
    """
    
    def __init__(self, files_by_product: Dict[str, Dict[datetime, Path]], 
                 temporal_window: int = 4):
        self.files = files_by_product
        self.window = temporal_window
        self._cache = {}  # (product, date) -> (data, lats, lons)
        self._grid_info = None  # (lats, lons) do primeiro arquivo carregado
    
    def _get_cache_key(self, product: str, date: datetime) -> Tuple:
        return (product, date)
    
    def _load_to_cache(self, product: str, date: datetime) -> bool:
        """Carrega dados para o cache. Retorna True se sucesso."""
        key = self._get_cache_key(product, date)
        if key in self._cache:
            return self._cache[key][0] is not None
        
        if product not in self.files or date not in self.files[product]:
            self._cache[key] = (None, None, None)
            return False
        
        filepath = self.files[product][date]
        config = PACE_PRODUCTS.get(product, {})
        var_name = config.get('var', product)
        
        # Para Kd, usar wavelength target
        wl_target = KD_TARGET_WAVELENGTH if product == 'kd' else None
        
        data, lats, lons = read_pace_variable(filepath, var_name, wl_target)
        self._cache[key] = (data, lats, lons)
        
        # Guardar info da grade
        if data is not None and self._grid_info is None:
            self._grid_info = (lats.copy(), lons.copy())
        
        return data is not None
    
    def get_value_strict(self, product: str, date: datetime, 
                         lat: float, lon: float) -> float:
        """
        Obtém valor interpolado para o dia exato.
        Retorna NaN se não houver dado no dia.
        """
        self._load_to_cache(product, date)
        data, lats, lons = self._cache.get(self._get_cache_key(product, date), (None, None, None))
        
        if data is None:
            return np.nan
        
        return self._interpolate(data, lats, lons, lat, lon)
    
    def get_value_filled(self, product: str, target_date: datetime,
                         lat: float, lon: float) -> Tuple[float, Optional[datetime]]:
        """
        Obtém valor interpolado, buscando em janela temporal se necessário.
        
        Returns:
            (valor, data_usada) - data_usada é None se o valor for NaN
        """
        if product not in self.files:
            return np.nan, None
        
        # Gerar lista de datas a tentar, ordenada por prioridade
        dates_to_try = self._get_dates_by_priority(target_date, self.files[product])
        
        for dt in dates_to_try:
            self._load_to_cache(product, dt)
            data, lats, lons = self._cache.get(self._get_cache_key(product, dt), (None, None, None))
            
            if data is None:
                continue
            
            val = self._interpolate(data, lats, lons, lat, lon)
            if np.isfinite(val):
                return val, dt
        
        return np.nan, None
    
    def _get_dates_by_priority(self, target: datetime, 
                               available: Dict[datetime, Path]) -> List[datetime]:
        """
        Retorna lista de datas ordenadas por prioridade.
        Prioridade: distância absoluta, depois passado em empate.
        """
        candidates = []
        for dt in available:
            delta = (dt - target).days
            if -self.window <= delta <= self.window:
                # Para ordenação: (distância_abs, delta positivo penalizado)
                # delta < 0 (passado) -> ordenar antes
                candidates.append((abs(delta), 0 if delta <= 0 else 1, dt))
        
        candidates.sort()
        return [c[2] for c in candidates]
    
    def _interpolate(self, data: np.ndarray, lats: np.ndarray, 
                     lons: np.ndarray, lat: float, lon: float) -> float:
        """Interpola bilinearmente o valor na posição lat/lon."""
        try:
            # Verificar bounds
            if lat < lats.min() or lat > lats.max():
                return np.nan
            if lon < lons.min() or lon > lons.max():
                return np.nan
            
            # Garantir ordem monotônica
            if lats[0] > lats[-1]:
                lats = lats[::-1]
                data = data[::-1, :]
            if lons[0] > lons[-1]:
                lons = lons[::-1]
                data = data[:, ::-1]
            
            interp = RegularGridInterpolator(
                (lats, lons), data, 
                method='linear',
                bounds_error=False, 
                fill_value=np.nan
            )
            return float(interp([[lat, lon]])[0])
        except Exception:
            return np.nan
    
    def clear_cache(self):
        """Limpa o cache para liberar memória."""
        self._cache.clear()


print("Classes de carregamento definidas.")

In [ ]:
# Cell 7: Função principal de extração

def extract_pace_variables(csv_path: str,
                           pace_files: Dict[str, Dict[datetime, Path]],
                           lat_col: str = 'LATITUDE_DEC',
                           lon_col: str = 'LONGITUDE_DEC', 
                           date_col: str = 'GMT_DATE_TIME',
                           temporal_window: int = 4,
                           products: List[str] = None) -> pd.DataFrame:
    """
    Extrai variáveis PACE para cada ponto do CSV.
    
    Para cada produto, gera duas colunas:
    - {produto}_strict: valor do dia exato (NaN se não disponível)
    - {produto}_filled: valor preenchido da janela temporal
    
    Args:
        csv_path: Caminho do CSV com dados de pesca
        pace_files: Dicionário de arquivos PACE por produto/data
        lat_col, lon_col, date_col: Nomes das colunas
        temporal_window: Janela em dias para preenchimento
        products: Lista de produtos a extrair (None = todos disponíveis)
    
    Returns:
        DataFrame com variáveis extraídas
    """
    # Carregar CSV
    print(f"\nCarregando CSV: {csv_path}")
    df = pd.read_csv(csv_path)
    print(f"  → {len(df)} registros")
    
    # Preparar colunas de coordenadas
    df['_lat'] = pd.to_numeric(df[lat_col], errors='coerce')
    df['_lon'] = pd.to_numeric(df[lon_col], errors='coerce')
    df['_date'] = pd.to_datetime(df[date_col], errors='coerce')
    
    # Filtrar registros válidos
    valid_mask = df['_lat'].notna() & df['_lon'].notna() & df['_date'].notna()
    df = df[valid_mask].reset_index(drop=True)
    print(f"  → {len(df)} registros com coordenadas válidas")
    
    # Produtos a processar
    if products is None:
        products = list(pace_files.keys())
    products = [p for p in products if p in pace_files]
    print(f"  → Produtos a extrair: {products}")
    
    # Inicializar loader
    loader = PACEDataLoader(pace_files, temporal_window)
    
    # Criar colunas de resultado
    n_rows = len(df)
    results = {}
    for product in products:
        output_name = PACE_PRODUCTS.get(product, {}).get('output_name', product)
        results[f'{output_name}_strict'] = np.full(n_rows, np.nan)
        results[f'{output_name}_filled'] = np.full(n_rows, np.nan)
        results[f'{output_name}_filled_date_offset'] = np.full(n_rows, np.nan)  # Dias de diferença
    
    # Extrair valores
    print("\nExtraindo valores PACE...")
    for product in products:
        output_name = PACE_PRODUCTS.get(product, {}).get('output_name', product)
        print(f"  Processando {product} → {output_name}...")
        
        for i in range(n_rows):
            if i % 1000 == 0 and i > 0:
                print(f"    {i}/{n_rows}...")
            
            lat = df.loc[i, '_lat']
            lon = df.loc[i, '_lon']
            date = df.loc[i, '_date'].to_pydatetime()
            
            # Valor strict (dia exato)
            val_strict = loader.get_value_strict(product, date, lat, lon)
            results[f'{output_name}_strict'][i] = val_strict
            
            # Valor filled (com janela temporal)
            val_filled, date_used = loader.get_value_filled(product, date, lat, lon)
            results[f'{output_name}_filled'][i] = val_filled
            
            if date_used is not None:
                offset = (date_used - date).days
                results[f'{output_name}_filled_date_offset'][i] = offset
        
        # Estatísticas
        n_strict = np.sum(np.isfinite(results[f'{output_name}_strict']))
        n_filled = np.sum(np.isfinite(results[f'{output_name}_filled']))
        print(f"    → strict: {n_strict}/{n_rows} ({100*n_strict/n_rows:.1f}%)")
        print(f"    → filled: {n_filled}/{n_rows} ({100*n_filled/n_rows:.1f}%)")
        
        # Limpar cache periódicamente para economizar memória
        loader.clear_cache()
    
    # Adicionar colunas ao DataFrame
    for col, values in results.items():
        df[col] = values
    
    # Calcular variáveis derivadas
    print("\nCalculando variáveis derivadas...")
    
    # Razão Chl:C (indicador de taxa de crescimento μ)
    if 'chlor_a_strict' in df.columns and 'carbon_phyto_strict' in df.columns:
        df['chl_c_ratio_strict'] = df['chlor_a_strict'] / df['carbon_phyto_strict']
        df['chl_c_ratio_filled'] = df['chlor_a_filled'] / df['carbon_phyto_filled']
        print("  → chl_c_ratio calculado")
    
    # Limpar colunas temporárias
    df = df.drop(columns=['_lat', '_lon', '_date'], errors='ignore')
    
    return df


print("Função de extração definida.")

In [ ]:
# Cell 8: Função para criar NetCDF consolidado (opcional)

def create_pace_daily_composite(pace_files: Dict[str, Dict[datetime, Path]],
                                 output_dir: Path,
                                 products: List[str] = None) -> None:
    """
    Cria arquivos NetCDF diários consolidados com todas as variáveis PACE.
    
    Isso facilita o uso no correlation_dashboard, que espera uma estrutura:
    output_dir/pace_composite_YYYYMMDD.nc
    
    Cada arquivo contém:
    - chlor_a
    - carbon_phyto
    - bbp_s
    - poc
    - Kd_490
    - chl_c_ratio (derivada)
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    if products is None:
        products = ['carbon', 'chl', 'iop', 'poc', 'kd']
    
    # Descobrir todas as datas disponíveis
    all_dates = set()
    for product in products:
        if product in pace_files:
            all_dates.update(pace_files[product].keys())
    all_dates = sorted(all_dates)
    
    print(f"\nCriando composites para {len(all_dates)} datas...")
    
    for date in all_dates:
        print(f"  {date.strftime('%Y-%m-%d')}...", end=" ")
        
        data_arrays = {}
        lats, lons = None, None
        
        for product in products:
            if product not in pace_files or date not in pace_files[product]:
                continue
            
            filepath = pace_files[product][date]
            config = PACE_PRODUCTS.get(product, {})
            var_name = config.get('var', product)
            output_name = config.get('output_name', product)
            
            wl_target = KD_TARGET_WAVELENGTH if product == 'kd' else None
            data, lat_arr, lon_arr = read_pace_variable(filepath, var_name, wl_target)
            
            if data is not None:
                data_arrays[output_name] = data
                if lats is None:
                    lats, lons = lat_arr, lon_arr
        
        if not data_arrays or lats is None:
            print("SKIP (sem dados)")
            continue
        
        # Calcular derivadas
        if 'chlor_a' in data_arrays and 'carbon_phyto' in data_arrays:
            with np.errstate(divide='ignore', invalid='ignore'):
                chl_c = data_arrays['chlor_a'] / data_arrays['carbon_phyto']
                chl_c = np.where(np.isfinite(chl_c), chl_c, np.nan)
            data_arrays['chl_c_ratio'] = chl_c
        
        # Criar Dataset xarray
        ds = xr.Dataset(
            {name: (['lat', 'lon'], arr) for name, arr in data_arrays.items()},
            coords={'lat': lats, 'lon': lons}
        )
        
        # Adicionar atributos
        ds.attrs['title'] = 'PACE OCI Daily Composite for Fishing Potential Analysis'
        ds.attrs['date'] = date.strftime('%Y-%m-%d')
        ds.attrs['source'] = 'NASA PACE OCI L3 products'
        ds.attrs['processing'] = 'Consolidated by pace_preprocessor.ipynb'
        
        # Atributos das variáveis
        var_attrs = {
            'chlor_a': {'long_name': 'Chlorophyll-a concentration', 'units': 'mg m^-3'},
            'carbon_phyto': {'long_name': 'Phytoplankton Carbon', 'units': 'mg m^-3'},
            'bbp_s': {'long_name': 'Backscattering spectral slope (eta)', 'units': 'dimensionless'},
            'poc': {'long_name': 'Particulate Organic Carbon', 'units': 'mg m^-3'},
            'Kd_490': {'long_name': 'Diffuse attenuation coefficient at 490nm', 'units': 'm^-1'},
            'chl_c_ratio': {'long_name': 'Chlorophyll:Carbon ratio (growth rate proxy)', 'units': 'mg Chl / mg C'},
        }
        for var in ds.data_vars:
            if var in var_attrs:
                ds[var].attrs.update(var_attrs[var])
        
        # Salvar
        outfile = output_dir / f"pace_composite_{date.strftime('%Y%m%d')}.nc"
        ds.to_netcdf(outfile)
        print(f"OK ({len(data_arrays)} vars)")
    
    print(f"\nComposites salvos em: {output_dir}")


print("Função de criação de composites definida.")

---
## Uso

### Opção 1: Extrair variáveis PACE para um CSV existente
Use quando quiser adicionar variáveis PACE diretamente a um arquivo de dados de pesca.

In [ ]:
# Cell 9: OPÇÃO 1 - Extrair para CSV
# Descomente e ajuste conforme necessário

# CSV_INPUT = "./data/fishing_data.csv"  # Seu arquivo de dados
# CSV_OUTPUT = "./data/fishing_data_with_pace.csv"

# # Extrair
# df_result = extract_pace_variables(
#     csv_path=CSV_INPUT,
#     pace_files=pace_files,
#     lat_col='LATITUDE_DEC',    # Ajustar conforme seu CSV
#     lon_col='LONGITUDE_DEC',   # Ajustar conforme seu CSV
#     date_col='GMT_DATE_TIME',  # Ajustar conforme seu CSV
#     temporal_window=4,
#     products=['carbon', 'chl', 'iop', 'poc', 'kd']  # Produtos desejados
# )

# # Salvar
# df_result.to_csv(CSV_OUTPUT, index=False)
# print(f"\nSalvo: {CSV_OUTPUT}")
# print(f"Colunas adicionadas: {[c for c in df_result.columns if 'strict' in c or 'filled' in c]}")

print("Opção 1 (extração para CSV) - descomente e configure acima para executar")

### Opção 2: Criar NetCDFs consolidados
Use quando quiser que o correlation_dashboard leia os dados PACE de uma pasta organizada.

In [ ]:
 Cell 10: OPÇÃO 2 - Criar composites NetCDF
 Descomente para executar

 create_pace_daily_composite(
     pace_files=pace_files,
     output_dir=OUTPUT_DIR,
     products=['carbon', 'chl', 'iop', 'poc', 'kd']
 )

print("Opção 2 (criar composites) - descomente acima para executar")

### Opção 3: Uso interativo / diagnóstico

In [ ]:
# Cell 11: Diagnóstico - verificar cobertura de dados

def diagnose_pace_coverage(pace_files: Dict[str, Dict[datetime, Path]],
                            test_dates: List[datetime] = None):
    """
    Mostra diagnóstico da cobertura de dados PACE.
    """
    print("\n" + "="*60)
    print("DIAGNÓSTICO DE COBERTURA PACE")
    print("="*60)
    
    all_dates = get_available_dates(pace_files)
    if not all_dates:
        print("Nenhum arquivo PACE encontrado!")
        return
    
    print(f"\nPeríodo coberto: {all_dates[0].strftime('%Y-%m-%d')} a {all_dates[-1].strftime('%Y-%m-%d')}")
    print(f"Total de datas únicas: {len(all_dates)}")
    
    # Cobertura por produto
    print("\nCobertura por produto:")
    for product, dates in sorted(pace_files.items()):
        coverage = len(dates) / len(all_dates) * 100 if all_dates else 0
        print(f"  {product:12s}: {len(dates):4d} dias ({coverage:5.1f}%)")
    
    # Testar algumas datas
    if test_dates:
        print("\nTeste de disponibilidade:")
        for dt in test_dates:
            print(f"  {dt.strftime('%Y-%m-%d')}:")
            for product in pace_files:
                status = "✓" if dt in pace_files[product] else "✗"
                print(f"    {product}: {status}")

# Executar diagnóstico
if pace_files:
    diagnose_pace_coverage(pace_files)
else:
    print("Nenhum arquivo PACE encontrado. Verifique o diretório configurado.")

In [ ]:
# Cell 12: Teste de extração pontual

def test_extraction(pace_files: Dict, 
                    test_lat: float, test_lon: float, 
                    test_date: datetime,
                    window: int = 4):
    """
    Testa extração de valores para um ponto específico.
    """
    print(f"\n" + "="*60)
    print(f"TESTE DE EXTRAÇÃO")
    print(f"="*60)
    print(f"Coordenadas: ({test_lat}, {test_lon})")
    print(f"Data: {test_date.strftime('%Y-%m-%d')}")
    print(f"Janela temporal: ±{window} dias")
    print()
    
    loader = PACEDataLoader(pace_files, window)
    
    for product in pace_files:
        output_name = PACE_PRODUCTS.get(product, {}).get('output_name', product)
        
        val_strict = loader.get_value_strict(product, test_date, test_lat, test_lon)
        val_filled, date_used = loader.get_value_filled(product, test_date, test_lat, test_lon)
        
        print(f"{output_name}:")
        print(f"  strict: {val_strict:.4f}" if np.isfinite(val_strict) else "  strict: NaN")
        if np.isfinite(val_filled):
            offset = (date_used - test_date).days if date_used else 0
            print(f"  filled: {val_filled:.4f} (offset: {offset:+d} dias)")
        else:
            print(f"  filled: NaN")
        print()

# Exemplo de teste (descomente e ajuste)
# test_extraction(
#     pace_files,
#     test_lat=-23.0,  # Exemplo: próximo a Arraial do Cabo
#     test_lon=-42.0,
#     test_date=datetime(2025, 3, 28),
#     window=4
# )

print("Função de teste definida. Descomente a chamada acima para testar.")

---
## Resumo das variáveis geradas

| Variável | Descrição | Relevância para Pesca |
|----------|-----------|----------------------|
| `chlor_a` | Clorofila-a [mg m⁻³] | Indicador tradicional de biomassa |
| `carbon_phyto` | C_phyto [mg m⁻³] | Biomassa real de carbono |
| `chl_c_ratio` | Chl:C | Proxy de taxa de crescimento μ (alto=crescimento ativo) |
| `bbp_s` | η (eta) | Inclinação PSD: baixo→partículas grandes→cadeia curta eficiente |
| `poc` | POC [mg m⁻³] | Carbono particulado total (inclui detritos) |
| `Kd_490` | Kd em 490nm [m⁻¹] | Atenuação → Z_eu = 4.6/Kd (profundidade eufótica) |

### Sufixos:
- `_strict`: Valor do dia exato (NaN se não disponível)
- `_filled`: Valor preenchido da janela ±4 dias
- `_filled_date_offset`: Dias de diferença do valor preenchido